# Implementation of [arXiv:1905.13729](https://arxiv.org/abs/1905.13729)
See also: DOI: [10.1103/PhysRevD.101.095032](http://doi.org/10.1103/PhysRevD.101.095032)
## General solution to the U(1) anomaly equations

Obtain a numpy array `z` of `N` integers which satisfy the Diophantine equations
$$ \sum_{i=1}^n z_i=0\,,\qquad \sum_{i=1}^n z_i^3=0\,,$$
such that
```python
>>> z.sum()
0
>>> (z**3).sum()
0
```
The input is two lists `l` and `k` with any `(N-3)/2` and `(N-1)/2` integers for `N` odd, or `N/2-1` and `N/2-1` for `N` even (`N>4`).
The function is implemented below under the name: `free(l,k)`

<a href="https://colab.research.google.com/github/restrepo/anomaly/blob/main/anomalysolutions-class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://stackoverflow.com/a/43793179/2268280

In [1]:
import pandas as pd
import numpy as np
from astropy.table import Table
import itertools
import sys
import os
from functools import reduce
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install anomalies 2>/dev/null > /dev/null
#if not os.path.exists('anomalytools.py'):
#    !wget https://raw.githubusercontent.com/restrepo/anomaly/main/anomalytools.py 2>/dev/null > /dev/null

In [3]:
from anomalies import anomaly
from anomalytools import *

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [5]:
anomaly.free([-1,1],[4,-2])

array([  3,   3,   3, -12, -12,  15])

In [6]:
anomaly.free.gcd

3

In [7]:
anomaly.free.simplified

array([ 1,  1,  1, -4, -4,  5])

In [8]:
z=anomaly.free

## Analysis
`solutions` class → Initialize the object to obtain anomaly free solutions for any set of `N` integers

In [9]:
#TODO: inherit from free class
import sys
def _get_chiral(q,q_max=np.inf):
    #Normalize to positive minimum
    if q[0]<0 or (q[0]==0 and q[1]<0):
        q=-q
    #Divide by GCD
    GCD=np.gcd.reduce(q)
    q=(q/GCD).astype(int)
    if ( #not 0 in z and 
          0 not in [ sum(p) for p in itertools.permutations(q, 2) ] and #avoid vector-like and multiple 0's
          #q.size > np.unique(q).size and # check for at least a duplicated entry
          np.abs(q).max()<=q_max
           ):
        return q,GCD
    else:
        return None,None
class solutions(object):
    '''
    Obtain anomaly free solutions with N chiral fields
    
    Call the initialize object with N and get the solutions:
    Example:
    >>> s=solutions()
    >>> s(6) # N = 6
    
    Redefine the self.chiral function to implement further restrictions:
    inherit from this class and define the new chiral function
    '''
    def __init__(self,nmin=-2,nmax=2,zmax=np.inf):
        self.nmin=nmin
        self.nmax=nmax
        self.zmax=zmax
        self.CALL=False

    def __call__(self,N,*args,**kwargs):
        self.CALL=True
        if N%2!=0: #odd
            N_l=(N-3)//2
            N_k=(N-1)//2
        else: #even
            N_l=N//2-1
            N_k=N_l
        r=range(self.nmin,self.nmax+1)
        self.ls=list(itertools.product( *(r for i in range(N_l)) ))
        self.ks=list(itertools.product( *(r for i in range(N_k)) ))
        return self.chiral(*args,**kwargs)
        
        
    def chiral(self,*args,**kwargs):
        if not self.CALL:
            sys.exit('Call the initialized object first:\n>>> s=solutions()\n>>> self(5)')
        self.list=[]
        solt=[]
        for l in self.ls:
            for k in self.ks:
                l=list(l)
                k=list(k)
                q,gcd=_get_chiral( z(l,k) )
                #print(z(l,k))
                if q is not None and list(q) not in self.list and list(-q) not in self.list:
                    self.list.append(list(q))
                    solt.append({'l':l,'k':k,'z':list(q),'gcd':gcd})
        return solt

Chiral solutions for l and k in the range [-2,2]

In [15]:
s=solutions()

solutions for $N=5$ integers

In [16]:
pd.DataFrame(  s(5)  )

,l,k,z,gcd
0,[-2],"[-1, 2]","[2, 4, -7, -9, 10]",1
1,[-2],"[2, -1]","[1, 5, -7, -8, 9]",4


In [17]:
class dark(solutions):
    '''
    Modify the self.chiral function to obtain solutions
    with either duplicate or triplicate integers
    '''
    def chiral(self,X=False,verbose=False,print_step=100000):
        m=2
        if X:
            m=3
        self.list=[]
        solt=[]
        tot=len(self.ls)*len(self.ks)
        i=0
        for l in self.ls:
            for k in self.ks:
                if verbose:
                    i=i+1
                    if i%print_step==0:
                        print('{}/{}'.format(i,tot))
                l=list(l)
                k=list(k)
                q,gcd=_get_chiral( z(l,k) )
                #print(z(l,k))
                if (q is not None and 
                    list(q) not in self.list and list(-q) not in self.list and
                    1 in [ len(set(p)) for p in itertools.permutations(q, m) ] and
                    #q.size-np.unique(q).size>m
                    np.abs(q).max()<=self.zmax
                   ):
                    self.list.append(list(q))
                    solt.append({'l':l,'k':k,'z':list(q),'gcd':gcd})
        return solt        

Chiral solutions with repeated integers

In [18]:
s=dark()

Example: Force solutions with triplicate integers

In [19]:
pd.DataFrame(   s(6,X=True)   )

,l,k,z,gcd
0,"[-2, 2]","[-2, -1]","[1, 1, 1, -4, -4, 5]",8


In [390]:
%%time
s=dark(nmin=-32,nmax=32)
s(5)

CPU times: user 19.1 s, sys: 12 ms, total: 19.1 s
Wall time: 19.1 s


[]

In [391]:
%%time
s=dark(nmin=-10,nmax=10,zmax=32)
s(6,X=True,verbose=True)

100000/194481
CPU times: user 18.3 s, sys: 12 ms, total: 18.3 s
Wall time: 18.3 s


[{'l': [-10, 5], 'k': [-2, 4], 'z': [1, 1, 1, -4, -4, 5], 'gcd': 1500},
 {'l': [-10, 5], 'k': [3, 4], 'z': [3, 3, 3, -10, -17, 18], 'gcd': 250}]

In [392]:
%%time
s=dark(nmin=-21,nmax=21,zmax=32)
s(6,X=True,verbose=True,print_step=500000)

500000/3418801
1000000/3418801
1500000/3418801
2000000/3418801
2500000/3418801
3000000/3418801
CPU times: user 5min 53s, sys: 35.9 ms, total: 5min 53s
Wall time: 5min 53s


[{'l': [-20, -10], 'k': [5, 10], 'z': [1, 1, 1, -4, -4, 5], 'gcd': 15000},
 {'l': [-20, 10], 'k': [6, 8], 'z': [3, 3, 3, -10, -17, 18], 'gcd': 4000}]

In [ ]:
%%time
s=dark(nmin=-21,nmax=21,zmax=32)
slt=s(6,verbose=True,print_step=500000)

In [398]:
pd.DataFrame(slt)

,l,k,z,gcd
0,"[-21, -18]","[-4, -7]","[3, -4, -21, 25, 25, -28]",7749
1,"[-21, -18]","[11, 14]","[9, -11, -16, 21, 21, -24]",3591
2,"[-21, -14]","[-21, 0]","[2, -3, -10, 13, 13, -15]",21609
3,"[-21, -14]","[-21, 3]","[4, -5, -11, 14, 14, -16]",21168
4,"[-21, -14]","[-14, -21]","[1, -2, -3, 5, 5, -6]",36015
5,"[-21, -12]","[9, -3]","[3, -7, -7, 13, 15, -17]",5508
6,"[-21, -12]","[9, 14]","[1, 6, 9, -21, -21, 26]",3213
7,"[-21, -7]","[-14, -21]","[2, 2, 3, -8, -12, 13]",7203
8,"[-21, -6]","[6, 3]","[2, -7, -7, 13, 22, -23]",2025
9,"[-21, -3]","[-21, -14]","[8, -13, -17, 27, 27, -32]",2646


## Appendix

In [433]:
ds=pd.read_json('solutions.json')
ds['gcd']=ds.apply( lambda row:  np.gcd.reduce( z(row['l'],row['k']) ) ,axis='columns' )

In [434]:
dsn6=ds[ds['n']==6].reset_index(drop=True)
dsn6[dsn6['solution'].apply(lambda q: 1 in [ len(set(p)) for p in itertools.permutations(q, 2) ] )].reset_index(drop=True)

,k,l,n,solution,gcd
0,"[-4, -3]","[-4, 4]",6,"[5, -9, -9, 16, 16, -19]",32
1,"[-2, -1]","[-2, 1]",6,"[4, -5, -11, 14, 14, -16]",1
2,"[4, -5]","[-5, -3]",6,"[10, -12, -12, 15, 17, -18]",45
3,"[7, -2]","[-3, 9]",6,"[1, 6, 9, -21, -21, 26]",108
4,"[9, -5]","[6, 21]",6,"[17, -18, -22, 25, 25, -27]",315
5,"[-3, -6]","[-3, 9]",6,"[8, -13, -13, 19, 31, -32]",162
6,"[12, -4]","[12, 6]",6,"[13, -17, -19, 27, 27, -31]",1152
7,"[-2, -5]","[-2, -1]",6,"[4, -10, -10, 17, 31, -32]",1
8,"[1, 3]","[6, -8]",6,"[3, 3, 3, -10, -17, 18]",168
9,"[11, 6]","[-9, 5]",6,"[9, -11, -16, 21, 21, -24]",330
